<a href="https://colab.research.google.com/github/dbtronics/COMP551---Miniproject-3-Classification-of-Image-Data-/blob/main/Group_18_Miniproject_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression

# MLP
Multilayer Perceptron Implementation

In [7]:
logistic = lambda z: 1./ (1 + np.exp(-z))
softmax = lambda z: np.exp(-z) / np.sum(np.exp(-z), axis = -1)[:, None]

class MLP:

    
  def __init__(self, M = 64):
    self.M = M
            
  def fit(self, x, y, optimizer, hidden_layer=1):
    N,D = x.shape
    C = np.max(y)+1
    y_realprob = np.zeros((N, C))
    for i in range(N): y_realprob[i, y[i]] = 1
    # START: Condition to have 0, 1, 2 layers
    # gradient(x,y,params, hidden_layers)
    def gradient_0layer(x, y, params):
      w = params
      yh = softmax(np.dot(x, w))
      dy = yh - y_realprob # Squared residual loss derivative
      # dw = np.dot(x.T, dy)
      dw = (x.T @ dy)/N
      dparams = [dw]


    def gradient_1layer(x, y, params):
      v, w = params
      z = logistic(np.dot(x, v)) #N x M
      yh = softmax(np.dot(z, w))#N x C
      # print(yh[i, np.argmax(yh[i,:])] for i in range(10))
      # print(y[:10])
      dy = yh - y_realprob #N x C
      dw = np.dot(z.T, dy)/N #M x C
      dz = np.dot(dy, w.T) #N x M
      # print(dz.shape)
      dv = np.dot(x.T, dz * z * (1 - z))/N #D x M
      dparams = [dv, dw]
      return dparams
    
    def gradient_2layer(x, y, params):
      return None
    gradient = [gradient_0layer, gradient_1layer, gradient_2layer]
        
    # params0 = [np.random.randn(self.M) for i in range(hidden_layer+1)]
    if(hidden_layer==0):
      w = np.random.randn(D, C) # weights b/w I/O
      params0 = [w]
    elif (hidden_layer==1):
      v = np.random.randn(D, self.M) # weights b/w I/first layer
      w = np.random.randn(self.M, C) # weights b/w first/second layer
      params0 = [v, w]
    elif (hidden_layer==2):
      v = np.random.randn(D, self.M) # weights b/w I/first layer
      r = np.random.randn(self.M, self.M) # weights b/w first/second layer
      w = np.random.randn(self.M, C) # weights b/w second/O layer
      params = [v, r, w]
    else: 
      print("hidden layer should be <= 2")
      return None
    # END: Condition to have 0, 1, 2 layers
    self.params = optimizer.run(gradient[hidden_layer], x, y, params0)
    return self
    
  def predict(self, x): # Tweak this method too
    v, w = self.params
    z = logistic(np.dot(x, v)) # N x M
    yh = logistic(np.dot(z, w)) # N
    return yh

In [ ]:
def eval_acc(y_pred, y_real):
  if(y_pred.ndim==1 and y_real.ndim==1): 
    return np.sum(y_pred==y_real)/len(y_pred)
  else: 
    print("dimension should be 1")
    return None

## Gradient Calculation

In [8]:
class GradientDescent:
    
  def __init__(self, learning_rate=.001, max_iters=1e4, epsilon=1e-8):
    self.learning_rate = learning_rate
    self.max_iters = max_iters
    self.epsilon = epsilon
        
  def run(self, gradient_fn, x, y, params):
    norms = np.array([np.inf])
    t = 1
    while np.any(norms > self.epsilon) and t < self.max_iters:
      # add parameter of number of layers
        grad = gradient_fn(x, y, params)
        for p in range(len(params)):
            params[p] -= self.learning_rate * grad[p]
        t += 1
        norms = np.array([np.linalg.norm(g) for g in grad])
        print("%d epoch completed with norm of:" %(t))
        print(norms)
    return params

# Main Body of Code
## Importing Data and normalization

In [9]:
mnist_dataset = tf.keras.datasets.fashion_mnist
(train_images, train_labels), (test_images, test_labels) = mnist_dataset.load_data()

print("Train image shape: ", train_images.shape)
print("Train label shape: ", train_labels.shape)
print("\nTest image shape: ", test_images.shape)
print("Test label shape: ", test_labels.shape)

train_images = train_images/255
test_images = test_images/255

# plt.imshow(train_images[10], cmap='gray')
# plt.colorbar()
# plt.grid(False)
# plt.show()

Train image shape:  (60000, 28, 28)
Train label shape:  (60000,)

Test image shape:  (10000, 28, 28)
Test label shape:  (10000,)


In [11]:
fraction = 0.10 # 10% validation and 90% training set
n_train = train_images.shape[0]
# w, h = train_images[0].shape
train_images = train_images[:int(fraction*n_train)]
train_labels = train_labels[:int(fraction*n_train)]
train_images = train_images.reshape(train_images.shape[0], -1)
print(train_images.shape)

n_train = train_images.shape[0]
x_test = train_images[:int(fraction*n_train)]
y_test = train_labels[:int(fraction*n_train)]

x_train = train_images[int(fraction*n_train):]
y_train = train_labels[int(fraction*n_train):]

mlp = MLP(128)
optimizer = GradientDescent()

mlp = mlp.fit(x_train, y_train, optimizer, hidden_layer=1)

Streaming output truncated to the last 5000 lines.
7501 epoch completed with norm of:
[2.04899708 8.47981128]
7502 epoch completed with norm of:
[2.04904998 8.47995724]
7503 epoch completed with norm of:
[2.04910355 8.48010323]
7504 epoch completed with norm of:
[2.0491578  8.48024924]
7505 epoch completed with norm of:
[2.04921273 8.48039526]
7506 epoch completed with norm of:
[2.04926833 8.4805413 ]
7507 epoch completed with norm of:
[2.0493246  8.48068737]
7508 epoch completed with norm of:
[2.04938155 8.48083346]
7509 epoch completed with norm of:
[2.04943917 8.48097956]
7510 epoch completed with norm of:
[2.04949746 8.48112569]
7511 epoch completed with norm of:
[2.04955643 8.48127184]
7512 epoch completed with norm of:
[2.04961606 8.481418  ]
7513 epoch completed with norm of:
[2.04967636 8.48156419]
7514 epoch completed with norm of:
[2.04973733 8.48171041]
7515 epoch completed with norm of:
[2.04979897 8.48185664]
7516 epoch completed with norm of:
[2.04986128 8.48200289]
7517 